In [9]:
from music21 import *
import glob
import pickle
import numpy as np
import os
from keras.models import load_model

In [16]:
midi = converter.parse('./archive/albeniz/alb_esp4.mid')

In [18]:
midi.show('text')

{0.0} <music21.stream.Part 0x2c9dc5db520>
    {0.0} <music21.instrument.Piano 'Piano right: Piano right'>
    {0.0} <music21.instrument.Piano 'Piano'>
    {0.0} <music21.tempo.MetronomeMark Quarter=111.0>
    {0.0} <music21.key.Key of B- major>
    {0.0} <music21.meter.TimeSignature 3/4>
    {0.0} <music21.stream.Voice 0x2c9dc5c3b80>
        {0.0} <music21.note.Note G>
        {0.25} <music21.note.Rest rest>
        {0.5} <music21.note.Note D>
        {0.75} <music21.note.Rest rest>
        {1.0} <music21.note.Note E->
        {1.25} <music21.note.Rest rest>
        {1.5} <music21.note.Note G>
        {1.75} <music21.note.Rest rest>
        {2.0} <music21.note.Note F#>
        {2.25} <music21.note.Rest rest>
        {2.5} <music21.note.Note A>
        {2.75} <music21.note.Rest rest>
        {3.0} <music21.note.Note G>
        {3.25} <music21.note.Note A>
        {3.5} <music21.note.Note F#>
        {3.75} <music21.note.Rest rest>
        {4.0} <music21.note.Note G>
        {4.25} <musi

## Preprocessing Midi File

In [44]:
notes = []

for folder in os.listdir('archive'):
    
    for file in glob.glob('archive/'+folder+'/*mid'):
        
        
        
        midi = converter.parse(file)
        print(f'Parsing File {file}')
        element_to_parse = midi.flat.notes


        for ele in element_to_parse:

            if isinstance(ele,note.Note):
                notes.append(str(ele.pitch))

            elif isinstance(ele,chord.Chord):
                notes.append('+'.join(str(n) for n in ele.normalOrder))
         
       

Parsing File archive/albeniz\alb_esp1.mid
Parsing File archive/albeniz\alb_esp2.mid
Parsing File archive/albeniz\alb_esp3.mid
Parsing File archive/albeniz\alb_esp4.mid
Parsing File archive/albeniz\alb_esp5.mid
Parsing File archive/albeniz\alb_esp6.mid
Parsing File archive/albeniz\alb_se1.mid
Parsing File archive/albeniz\alb_se2.mid
Parsing File archive/albeniz\alb_se3.mid
Parsing File archive/albeniz\alb_se4.mid
Parsing File archive/albeniz\alb_se5.mid
Parsing File archive/albeniz\alb_se6.mid
Parsing File archive/albeniz\alb_se7.mid
Parsing File archive/albeniz\alb_se8.mid
Parsing File archive/bach\bach_846.mid
Parsing File archive/bach\bach_847.mid
Parsing File archive/bach\bach_850.mid
Parsing File archive/balakir\islamei.mid
Parsing File archive/beeth\appass_1.mid
Parsing File archive/beeth\appass_2.mid
Parsing File archive/beeth\appass_3.mid
Parsing File archive/beeth\beethoven_hammerklavier_1.mid
Parsing File archive/beeth\beethoven_hammerklavier_2.mid
Parsing File archive/beeth\b

Parsing File archive/mendelssohn\mendel_op19_4.mid
Parsing File archive/mendelssohn\mendel_op19_5.mid
Parsing File archive/mendelssohn\mendel_op19_6.mid
Parsing File archive/mendelssohn\mendel_op30_1.mid
Parsing File archive/mendelssohn\mendel_op30_2.mid
Parsing File archive/mendelssohn\mendel_op30_3.mid
Parsing File archive/mendelssohn\mendel_op30_4.mid
Parsing File archive/mendelssohn\mendel_op30_5.mid
Parsing File archive/mendelssohn\mendel_op53_5.mid
Parsing File archive/mendelssohn\mendel_op62_3.mid
Parsing File archive/mendelssohn\mendel_op62_4.mid
Parsing File archive/mendelssohn\mendel_op62_5.mid
Parsing File archive/mozart\mz_311_1.mid
Parsing File archive/mozart\mz_311_2.mid
Parsing File archive/mozart\mz_311_3.mid
Parsing File archive/mozart\mz_330_1.mid
Parsing File archive/mozart\mz_330_2.mid
Parsing File archive/mozart\mz_330_3.mid
Parsing File archive/mozart\mz_331_1.mid
Parsing File archive/mozart\mz_331_2.mid
Parsing File archive/mozart\mz_331_3.mid
Parsing File archiv

In [45]:
with open('notes.pkl','wb') as f:
    pickle.dump(notes,f)

In [4]:
len(notes)

519995

In [3]:
vocab_size = len(set(notes))

In [49]:
vocab_size

483

In [2]:
with open('notes.pkl','rb') as f:
    notes = pickle.load(f)

In [6]:
pitchnames = sorted(set(notes))

In [7]:
ele_to_int = dict((ele,num) for num,ele in enumerate(pitchnames))

In [28]:
sequence_length=100
network_input = []

for i in range(len(notes)-sequence_length):
    seq_in = notes[i:i+sequence_length]
    network_input.append([ele_to_int[ch] for ch in seq_in])

In [11]:
model = load_model('model.hdf5')

In [31]:
start = np.random.randint(len(network_input)-1)
int_to_ele = dict((num,ele) for num,ele in enumerate(pitchnames))

pattern = network_input[start]
prediction_output = []


for note_index in range(200):
    prediction_input = np.reshape(pattern,(1,len(pattern),1))
    prediction_input = prediction_input/float(vocab_size)
    
    prediction = model.predict(prediction_input,verbose=0)
    
    idx = np.argmax(prediction)
    result = int_to_ele[idx]
    prediction_output.append(result)
    
    pattern.append(idx)
    pattern = pattern[1:]
    
    

In [34]:
print(prediction_output)

['G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4

In [35]:
offset = 0 # Time
output_notes = []

for pattern in prediction_output:
    
    # if the pattern is a chord
    if ('+' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('+')
        temp_notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))  # create Note object for each note in the chord
            new_note.storedInstrument = instrument.Piano()
            temp_notes.append(new_note)
            
        
        new_chord = chord.Chord(temp_notes) # creates the chord() from the list of notes
        new_chord.offset = offset
        output_notes.append(new_chord)
    
    else:
            # if the pattern is a note
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        
    offset += 0.5

In [36]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp = "test_output.mid")

'test_output.mid'

In [43]:
midi_stream.show('midi')